In [27]:
### original Bert for rule interestingness classification

In [28]:
import sys

datasetName = sys.argv[1] #'airports' # hospital, ncvoter, inspection
mlmOption = sys.argv[2]
result_file = sys.argv[3]

import os
import pandas as pd
import numpy as np
import copy

model_checkpoint_ = "distilbert-base-uncased"
mlm_model_dir = os.path.join('./saved_models/', datasetName + '-' + model_checkpoint_ + '-mlm')
saved_model_path = os.path.join('./saved_models/', datasetName + '-' + model_checkpoint_ + '-finetune')

model_checkpoint = copy.deepcopy(model_checkpoint_)
if mlmOption == True:
    model_checkpoint = mlm_model_dir #"distilbert-base-uncased"
data_prefix = '../../REEs_model_data/revision/labeled_data/'
rules_set_path = os.path.join(data_prefix, datasetName, datasetName + '_rules.txt')

In [29]:
# process rules

data_prefix = '../../REEs_model_data/revision/labeled_data/'
rules_set_path = os.path.join(data_prefix, datasetName, datasetName + '_rules.txt')

rules_set = pd.read_csv(rules_set_path)
rules_set = rules_set.values

for option in ('train', 'valid', 'test'):
    rule_pair_set_path = os.path.join(data_prefix, datasetName, 'train', option + '.csv')
    rules_pair_ids_set = pd.read_csv(rule_pair_set_path, delimiter=",")
    #rules_pair_ids_set
    # split training, validation and testing
    # prepare the rule pairs
    rules_pairs_set = []
    for left_id, right_id, label in rules_pair_ids_set.values:
        rule_left, rule_right = rules_set[left_id][0], rules_set[right_id][0]
        rules_pairs_set.append([rule_left, rule_right, label])
    rules_pairs_set = np.array(rules_pairs_set)
    #rules_pairs_set[:10]
    # save csv
    rules_pairs_df = pd.DataFrame(rules_pairs_set, columns=['rule_left', 'rule_right', 'label'])
    output_file = os.path.join(data_prefix, datasetName, datasetName + '_rule_pairs_' + option + '.csv')
    rules_pairs_df.to_csv(output_file, index=False)

In [30]:
# set gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [31]:
import transformers

from datasets import load_metric

In [32]:
from datasets import load_dataset

#dataset = load_dataset('csv', data_files='./top-k/rules/test_rules.csv')
_datasets = []
for option in ('train', 'valid', 'test'):
    rule_pairs_file = os.path.join(data_prefix, datasetName, datasetName + '_rule_pairs_' + option + '.csv')
    tmp_dataset = load_dataset('csv', data_files=rule_pairs_file)
    _datasets.append(tmp_dataset)

train_dataset, valid_dataset, test_dataset = _datasets

Using custom data configuration default-0054110a431d2cf8


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/yaoshuw/.cache/huggingface/datasets/csv/default-0054110a431d2cf8/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


Using custom data configuration default-ceab8322d7fc6a4a


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/yaoshuw/.cache/huggingface/datasets/csv/default-ceab8322d7fc6a4a/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


Using custom data configuration default-20b7339da673323a


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/yaoshuw/.cache/huggingface/datasets/csv/default-20b7339da673323a/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


In [33]:
train_dataset['train']['rule_left']

['airports(t0) ^ airports(t1) ^ t0.gps_code == t1.gps_code ^ t0.iso_country == t1.iso_country ^ t0.municipality == t1.municipality  ->  t0.local_code == t1.local_code',
 'airports(t0) ^ airports(t1) ^ t0.latitude_deg == t1.latitude_deg ^ t0.local_code == t1.local_code  ->  t0.continent == t1.continent',
 'airports(t0) ^ airports(t1) ^ t0.home_link == t1.home_link ^ t0.iso_country == BR  ->  t0.iso_country == t1.iso_country',
 'airports(t0) ^ airports(t1) ^ t0.iso_country == BR ^ t0.scheduled_service == t1.scheduled_service  ->  t0.wikipedia_link == t1.wikipedia_link',
 'airports(t0) ^ airports(t1) ^ t0.local_code == t1.local_code ^ t0.longitude_deg == t1.longitude_deg  ->  t0.type == t1.type',
 'airports(t0) ^ airports(t1) ^ t0.iso_region == t1.iso_region ^ t0.latitude_deg == t1.latitude_deg  ->  t0.type == t1.type',
 'airports(t0) ^ airports(t1) ^ t0.iso_country == US ^ t0.type == t1.type  ->  t0.iso_country == t1.iso_country',
 'airports(t0) ^ airports(t1) ^ t0.elevation_ft == t1.ele

In [34]:
dataset = train_dataset
dataset['validation'] = valid_dataset['train']
dataset['test'] = test_dataset['train']
# dataset['train'] = dataset['train']
dataset

DatasetDict({
    train: Dataset({
        features: ['rule_left', 'rule_right', 'label'],
        num_rows: 479
    })
    validation: Dataset({
        features: ['rule_left', 'rule_right', 'label'],
        num_rows: 159
    })
    test: Dataset({
        features: ['rule_left', 'rule_right', 'label'],
        num_rows: 161
    })
})

In [35]:
from transformers import AutoTokenizer
batch_size = 16

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_, use_fast=True)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/yaoshuw/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.12.2",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /home/yaoshuw/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79b

In [36]:
rule1_key, rule2_key = 'rule_left', 'rule_right'

dataset['train'][0][rule1_key]

'airports(t0) ^ airports(t1) ^ t0.gps_code == t1.gps_code ^ t0.iso_country == t1.iso_country ^ t0.municipality == t1.municipality  ->  t0.local_code == t1.local_code'

In [37]:
def preprocess_function(examples):
    return tokenizer(examples[rule1_key], examples[rule2_key], truncation=True)

In [38]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[101, 13586, 1006, 1056, 2692, 1007, 1034, 13586, 1006, 1056, 2487, 1007, 1034, 1056, 2692, 1012, 14658, 1035, 3642, 1027, 1027, 1056, 2487, 1012, 14658, 1035, 3642, 1034, 1056, 2692, 1012, 11163, 1035, 2406, 1027, 1027, 1056, 2487, 1012, 11163, 1035, 2406, 1034, 1056, 2692, 1012, 3250, 1027, 1027, 1056, 2487, 1012, 3250, 1011, 1028, 1056, 2692, 1012, 2334, 1035, 3642, 1027, 1027, 1056, 2487, 1012, 2334, 1035, 3642, 102, 13586, 1006, 1056, 2692, 1007, 1034, 13586, 1006, 1056, 2487, 1007, 1034, 1056, 2692, 1012, 16948, 1035, 4957, 1027, 1027, 1056, 2487, 1012, 16948, 1035, 4957, 1011, 1028, 1056, 2692, 1012, 9833, 1035, 3642, 1027, 1027, 1056, 2487, 1012, 9833, 1035, 3642, 102], [101, 13586, 1006, 1056, 2692, 1007, 1034, 13586, 1006, 1056, 2487, 1007, 1034, 1056, 2692, 1012, 15250, 1035, 2139, 2290, 1027, 1027, 1056, 2487, 1012, 15250, 1035, 2139, 2290, 1034, 1056, 2692, 1012, 2334, 1035, 3642, 1027, 1027, 1056, 2487, 1012, 2334, 1035, 3642, 1011, 1028, 1056, 2692, 1012, 

In [39]:
#tokenizer.decode(_['input_ids'][0])

'[CLS] airports ( t0 ) ^ airports ( t1 ) ^ t0. gps _ code = = t1. gps _ code ^ t0. iso _ country = = t1. iso _ country ^ t0. municipality = = t1. municipality - > t0. local _ code = = t1. local _ code [SEP] airports ( t0 ) ^ airports ( t1 ) ^ t0. wikipedia _ link = = t1. wikipedia _ link - > t0. iata _ code = = t1. iata _ code [SEP]'

In [40]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 479
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 159
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 161
    })
})

In [41]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 479
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 159
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 161
    })
})

In [42]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

loading configuration file ./saved_models/airports-distilbert-base-uncased-mlm/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.12.2",
  "vocab_size": 30522
}

loading weights file ./saved_models/airports-distilbert-base-uncased-mlm/pytorch_model.bin
Some weights of the model checkpoint at ./saved_models/airports-distilbert-base-uncased-mlm were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias'

In [43]:
metric = 'accuracy'
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(output_dir=saved_model_path,
                        evaluation_strategy='epoch',
                        save_strategy='epoch',
                        learning_rate=2e-5,
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        num_train_epochs=20,
                        weight_decay=0.01,
                        load_best_model_at_end=True,
                        metric_for_best_model='accuracy',
                        push_to_hub=False)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [44]:

# >>> metrics.precision_score(y_true, y_pred)
# 1.0
# >>> metrics.recall_score(y_true, y_pred)
# 0.5
# >>> metrics.f1_score(y_true, y_pred)
# accuracy_score(y_true, y_pred)
from sklearn import metrics
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    #print(predictions, labels)
    precision = metrics.precision_score(labels, predictions)
    recall = metrics.recall_score(labels, predictions)
    f1 = metrics.f1_score(labels, predictions)
    acc = metrics.accuracy_score(labels, predictions)
    return {'precision': precision, 'recall': recall, 'f1': f1, 'accuracy': acc}


trainer = Trainer(model, args, train_dataset=encoded_dataset['train'], eval_dataset=encoded_dataset['validation'], tokenizer=tokenizer, compute_metrics=compute_metrics)


In [45]:

os.environ["WANDB_DISABLED"] = "true"

In [46]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: rule_right, rule_left.
***** Running training *****
  Num examples = 479
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 600


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.699508,0.000000,0.000000,0.000000,0.547170
2,No log,0.687986,0.000000,0.000000,0.000000,0.547170
3,No log,0.694684,0.452830,1.000000,0.623377,0.452830
4,No log,0.680987,0.647059,0.152778,0.247191,0.578616
5,No log,0.655645,0.674419,0.402778,0.504348,0.641509
6,No log,0.640909,0.588889,0.736111,0.654321,0.647799
7,No log,0.647971,0.623188,0.597222,0.609929,0.654088
8,No log,0.660973,0.632353,0.597222,0.614286,0.660377
9,No log,0.669832,0.623188,0.597222,0.609929,0.654088
10,No log,0.688262,0.583333,0.777778,0.666667,0.647799


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: rule_right, rule_left.
***** Running Evaluation *****
  Num examples = 159
  Batch size = 16
/home/yaoshuw/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./saved_models/airports-distilbert-base-uncased-finetune/checkpoint-30
Configuration saved in ./saved_models/airports-distilbert-base-uncased-finetune/checkpoint-30/config.json
Model weights saved in ./saved_models/airports-distilbert-base-uncased-finetune/checkpoint-30/pytorch_model.bin
tokenizer config file saved in ./saved_models/airports-distilbert-base-uncased-finetune/checkpoint-30/tokenizer_config.json
Special tok

TrainOutput(global_step=600, training_loss=0.5029336992899577, metrics={'train_runtime': 115.5324, 'train_samples_per_second': 82.92, 'train_steps_per_second': 5.193, 'total_flos': 324898286861820.0, 'train_loss': 0.5029336992899577, 'epoch': 20.0})

In [47]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: rule_right, rule_left.
***** Running Evaluation *****
  Num examples = 159
  Batch size = 16


{'eval_loss': 0.6834944486618042,
 'eval_precision': 0.6461538461538462,
 'eval_recall': 0.5833333333333334,
 'eval_f1': 0.6131386861313869,
 'eval_accuracy': 0.6666666666666666,
 'eval_runtime': 0.2499,
 'eval_samples_per_second': 636.274,
 'eval_steps_per_second': 40.017,
 'epoch': 20.0}

In [48]:
ll = trainer.evaluate(encoded_dataset['test'])
print(ll)

f = open(result_file, 'w')
f.write(str(ll))
f.close()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: rule_right, rule_left.
***** Running Evaluation *****
  Num examples = 161
  Batch size = 16


{'eval_loss': 0.6444650292396545, 'eval_precision': 0.6428571428571429, 'eval_recall': 0.6338028169014085, 'eval_f1': 0.6382978723404257, 'eval_accuracy': 0.6832298136645962, 'eval_runtime': 0.2617, 'eval_samples_per_second': 615.321, 'eval_steps_per_second': 42.041, 'epoch': 20.0}


In [49]:
# save model
trainer.save_model()

Saving model checkpoint to ./saved_models/airports-distilbert-base-uncased-finetune
Configuration saved in ./saved_models/airports-distilbert-base-uncased-finetune/config.json
Model weights saved in ./saved_models/airports-distilbert-base-uncased-finetune/pytorch_model.bin
tokenizer config file saved in ./saved_models/airports-distilbert-base-uncased-finetune/tokenizer_config.json
Special tokens file saved in ./saved_models/airports-distilbert-base-uncased-finetune/special_tokens_map.json
